In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Clear_Cell_Carcinoma"
cohort = "GSE150404"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma/GSE150404"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/GSE150404.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE150404.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE150404.csv"
json_path = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from patients with different stages of clear cell renal cell carcinoma"
!Series_summary	"Clear cell renal cell carcinoma is the most common type among kidney cancers, accounting for 80% of all kidney cancer cases and over 30% of patients have metastases at the time disease is diagnosed. The process od epithelial-mesenchymal transition is crucial for the development and progression of cancer, and here we wanted to check the expression of genes on different stages of tumor progression, and find which of them changes significantly with increased grade."
!Series_overall_design	"Tumor samples from patients were divided into four grades by hospital pathologists. I- grade refers to the first stage of disease, II- grade to second, III-grade to third and IV-grade to fourth. Each patients received unrepetable number. RNA extraction were performed using Eurx kit, and then microarrays were performed using Affymetrix GeneChip WT PLUS Reagent Ki

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Assessment of Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from clear cell renal cell carcinoma patients
# The Series_overall_design mentions microarrays using Affymetrix GeneChip WT PLUS Reagent Kit, which indicates gene expression data
is_gene_available = True

# 2. Variable Availability and Type Conversion

# 2.1 Trait data (clear cell renal cell carcinoma stage)
# The trait row (Stage) is found at index 1 in the sample characteristics dictionary
trait_row = 1

# Function to convert trait values (Stage of cancer)
def convert_trait(value):
    # The value format is 'Stage: value'
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Convert stages to binary (early vs late stage)
    if value in ['first', 'second']:
        return 0  # Early stage (I-II)
    elif value in ['third', 'fourth']:
        return 1  # Late stage (III-IV)
    else:
        return None

# 2.2 Age data
# Age data is at index 3
age_row = 3

# Function to convert age values
def convert_age(value):
    # The value format is 'age: value'
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value == 'no data':
        return None
    
    try:
        return float(value)  # Convert to numeric (continuous)
    except:
        return None

# 2.3 Gender data
# Gender data is at index 2
gender_row = 2

# Function to convert gender values
def convert_gender(value):
    # The value format is 'gender: value'
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    elif value == 'no data':
        return None
    else:
        return None

# 3. Save Metadata - initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save the clinical data to CSV
    # First ensure the directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'GSM4548605': [0.0, 55.0, 0.0], 'GSM4548606': [0.0, 77.0, 1.0], 'GSM4548607': [0.0, 63.0, 0.0], 'GSM4548608': [0.0, 79.0, 0.0], 'GSM4548609': [0.0, 65.0, 0.0], 'GSM4548610': [0.0, 75.0, 0.0], 'GSM4548611': [0.0, 70.0, 0.0], 'GSM4548612': [0.0, 60.0, 0.0], 'GSM4548613': [0.0, 70.0, 1.0], 'GSM4548614': [0.0, 65.0, 0.0], 'GSM4548615': [0.0, 56.0, 0.0], 'GSM4548616': [0.0, 74.0, 0.0], 'GSM4548617': [0.0, 60.0, 1.0], 'GSM4548618': [0.0, 61.0, 1.0], 'GSM4548619': [0.0, 83.0, 0.0], 'GSM4548620': [0.0, 81.0, 0.0], 'GSM4548621': [0.0, 62.0, 0.0], 'GSM4548622': [0.0, 63.0, 1.0], 'GSM4548623': [0.0, 65.0, 1.0], 'GSM4548624': [0.0, 61.0, 1.0], 'GSM4548625': [0.0, 73.0, 1.0], 'GSM4548626': [0.0, 63.0, 0.0], 'GSM4548627': [0.0, 64.0, 0.0], 'GSM4548628': [0.0, 66.0, 1.0], 'GSM4548629': [0.0, 64.0, 0.0], 'GSM4548630': [0.0, 72.0, 0.0], 'GSM4548631': [0.0, 52.0, 1.0], 'GSM4548632': [0.0, 61.0, 0.0], 'GSM4548633': [0.0, 67.0, 1.0], 'GSM4548634': [0.0, 71.0, 1.0], 'GSM4548635': [1

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 53617 rows
First 20 gene IDs:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be numeric IDs rather than standard human gene symbols
# These appear to be probe IDs from a microarray platform which need to be mapped
# to standard human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 3270697 rows

Gene annotation preview (first few rows):
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box h

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the gene annotation data contain the IDs and gene symbols
# From the previous output, we can see the gene identifiers are numeric IDs like '16650001'
# Looking at the gene_annotation preview, 'ID' is the column containing these probe IDs
# For gene symbols, we need to extract from the 'gene_assignment' column

print("Step 1: Creating gene mapping from annotation data")
# 2. Extract relevant columns for mapping
try:
    # The 'ID' column in gene_annotation contains probe IDs that match gene_data index
    # The 'gene_assignment' column contains gene information with symbols
    mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')
    print(f"Successfully created gene mapping with {len(mapping_df)} rows")
    
    # Preview the mapping data
    print("\nFirst few rows of the gene mapping:")
    print(mapping_df.head())
    
    # 3. Apply the mapping to convert probe-level data to gene-level data
    print("\nStep 2: Converting probe measurements to gene expression data")
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    print(f"Successfully mapped probes to genes. Gene expression data has {len(gene_data)} genes and {gene_data.shape[1]} samples")
    
    # Preview the gene expression data
    print("\nFirst few gene symbols in the gene expression data:")
    print(gene_data.index[:5])
    
    # Save the gene data to file 
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"\nGene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error in gene mapping: {e}")
    is_gene_available = False


Step 1: Creating gene mapping from annotation data
Successfully created gene mapping with 53617 rows

First few rows of the gene mapping:
         ID                                               Gene
0  16657436  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  16657440  ENST00000473358 // MIR1302-11 // microRNA 1302...
2  16657445  NM_001005484 // OR4F5 // olfactory receptor, f...
3  16657447                                                ---
4  16657450  AK302511 // LOC100132062 // uncharacterized LO...

Step 2: Converting probe measurements to gene expression data


Successfully mapped probes to genes. Gene expression data has 81076 genes and 60 samples

First few gene symbols in the gene expression data:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I'], dtype='object', name='Gene')



Gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE150404.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Check if gene_data is empty before normalization
    if gene_data.empty:
        print("Gene data is empty. Skipping normalization.")
        normalized_gene_data = gene_data
    else:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data (even if empty, to maintain consistent workflow)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# Load the clinical data from the previously saved file
print("\nLoading clinical data...")
try:
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    is_trait_available = True
except Exception as e:
    print(f"Error loading clinical data: {e}")
    clinical_df = pd.DataFrame()
    is_trait_available = False

# 2. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not normalized_gene_data.empty:
        # Print sample IDs from both datasets for debugging
        print("Clinical data index:", clinical_df.index.tolist())
        print("First few clinical sample columns:", list(clinical_df.columns)[:5])
        print("First few genetic sample columns:", list(normalized_gene_data.columns)[:5])
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # After linking, the trait should be a column (not a row)
        # Check if the trait name is in the columns or if we need to use the first row of clinical data
        print(f"Linked data columns: {linked_data.columns[:10].tolist()}")
        
        # In the linked data, the trait is the first index from clinical_df
        actual_trait_name = clinical_df.index[0]
        print(f"Actual trait column name: {actual_trait_name}")
        
        if actual_trait_name in linked_data.columns:
            # 3. Handle missing values systematically
            print("\nHandling missing values...")
            linked_data = handle_missing_values(linked_data, actual_trait_name)
            print(f"After handling missing values, data shape: {linked_data.shape}")
            
            # Check if we still have samples after missing value handling
            if linked_data.shape[0] > 0:
                # 4. Determine whether the trait and demographic features are biased
                print("\nChecking for bias in features...")
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, actual_trait_name)
            else:
                print("Error: All samples were removed during missing value handling.")
                is_biased = True
        else:
            print(f"Trait column '{actual_trait_name}' not found in linked data.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing.")
        if not is_trait_available:
            print("Reason: Trait data is not available.")
        if normalized_gene_data.empty:
            print("Reason: Gene expression data is empty.")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
note = ""
if 'linked_data' not in locals() or linked_data.empty:
    note = "Dataset failed preprocessing: Unable to create usable linked data. "
    if not is_trait_available:
        note += "Clinical trait data was not available or could not be loaded."
    elif normalized_gene_data.empty:
        note += "Gene expression data was empty after processing."
    else:
        note += "The clinical data and gene data were available but could not be properly linked or processed."

is_gene_available = not normalized_gene_data.empty if 'normalized_gene_data' in locals() else False

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 6. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 23274 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE150404.csv

Loading clinical data...
Loaded clinical data with shape: (3, 60)

Linking clinical and genetic data...
Clinical data index: ['Kidney_Clear_Cell_Carcinoma', 'Age', 'Gender']
First few clinical sample columns: ['GSM4548605', 'GSM4548606', 'GSM4548607', 'GSM4548608', 'GSM4548609']
First few genetic sample columns: ['GSM4548605', 'GSM4548606', 'GSM4548607', 'GSM4548608', 'GSM4548609']
Linked data shape: (60, 23277)
Linked data columns: ['Kidney_Clear_Cell_Carcinoma', 'Age', 'Gender', 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1']
Actual trait column name: Kidney_Clear_Cell_Carcinoma

Handling missing values...


After handling missing values, data shape: (60, 23277)

Checking for bias in features...
For the feature 'Kidney_Clear_Cell_Carcinoma', the least common label is '0.0' with 30 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Kidney_Clear_Cell_Carcinoma' in this dataset is fine.

Quartiles for 'Age':
  25%: 58.75
  50% (Median): 63.824561403508774
  75%: 69.0
Min: 45.0
Max: 83.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 26 occurrences. This represents 43.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Performing final validation...


Linked data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/GSE150404.csv
